In [1]:
from Analys_data import Analys
import pandas as pd
import pickle


import torch 
from torch.utils.data import DataLoader
from sklearn.decomposition import PCA
import numpy as np

import sys
print(sys.path.append("/data/hdd1/brain/BraTS19/YandexCup/sentence-transformers"))
from sentence_transformers import SentenceTransformer


None


In [2]:
#1 кластеризация из топ n в BOW [SentenceTransformer -> AgglomerativeClustering -> KNeighborsClassifier ]+
#2 кластеризация 500-700 тысяч семплов +


In [2]:
support = Analys("data/metadata.json")

5462418it [00:24, 225521.06it/s]


In [15]:
n_components = 100
pca_model =  PCA(n_components = n_components, random_state=32)

def apply_pca(embedings):
    n = embedings.shape[0]
    print(embedings.reshape(n,-1).shape)
    main_class_emb = pca_model.fit_transform(embedings.reshape(n,-1))
    return main_class_emb, pca_model
     
def mean_tensor(embedings,class_count):
    
    """
    Сюда могут залететь нулевый классы class_count[i] == 0
    """
    step = [0,0]
    new_emb = torch.zeros((len(class_count),embedings.shape[1]),)
    for i, cl_c in enumerate(class_count):
        if cl_c == 0:
            continue
        step[1] += cl_c
        new_emb[i] = torch.mean(embedings[step[0]:step[1]], 0)
        step[0] += cl_c
    return new_emb
        
def data_to_claster(data, model_clastre, bs):
    # token convert to embedding
    
    embedder = SentenceTransformer('all-MiniLM-L6-v2')
    dataset_emb = DataLoader(embedder, batch_size=bs)
    
    embedings = torch.zeros((dataset_emb.shape[0],4,768))
    step = [0, 0]
    for sents in dataset:
        sents_g = sents.cuda()
        step[1] += bs
        embedings[step[0]:step[1]] = model.bert.embeddings(sents_g).cpu().detach()
        step[0] += bs
        del sents_g
        
    embedings = mean_tensor(embedings,data[1])
    embedings = apply_pca(embedings)
    predicts = model_clastre.predict(embedings)
    return predicts
    

In [3]:
data_start, idx = support.get_random_data(30)
count_class = [len(sentes) for sentes in data_start]
data = list(chain.from_iterable(data_start))

NameError: name 'data' is not defined

In [ ]:
pred = data_to_claster(data, model_clastre,4)

In [ ]:
### second part 

In [4]:
from Visual_embeddings import VisualEmb
import pickle
from Preprocessing_text import PreprocText,config_prep_text

In [5]:
# all-MiniLM-L6-v2 -> model

In [33]:
from sentence_transformers import SentenceTransformer
import numpy as np
import random
from itertools import chain
embedder = SentenceTransformer('all-MiniLM-L6-v2')
with open("model/KnnClassif.pkl","rb") as m:
    clust_classif_model = pickle.load(m)

In [8]:
from itertools import chain

In [5]:
config_prep_text["min_len_sent"] = 3

In [6]:
config_prep_text["clean_stw"] = False
config_prep_text["clean_hashtag"] = False

config_prep_text["clean_stw"] = False
config_prep_text["clean_link"] = False

In [3]:
import pandas as pd

In [ ]:
data_start,idx = support.get_random_data(700000)
count_class = [len(sentes) for sentes in data_start]
data = list(chain.from_iterable(data_start))


cfg = config_prep_text
processer = PreprocText(config = cfg)
# если длина строки меньше 1 слова то мы ее выкидываем
data_proc = processer.processing_big_data(data,class_count = count_class.copy(), n_worker = 16)
c_empty_cls = processer.empty_class # эти классы выкидываем из предиктов

In [32]:
# Corpus with example sentences

corpus_embeddings = embedder.encode(data_proc[0])

# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

#means tesor
corpus_embeddings = mean_tensor(torch.tensor(corpus_embeddings),data_proc[1])

# Perform kmean clustering
cluster_assignment = clust_classif_model.predict(corpus_embeddings) # 

NameError: name 'embedder' is not defined

In [235]:
clusters = {}
clusters_count = {}
for i, pred in enumerate(cluster_assignment):
    if i in c_empty_cls:
        continue
        
    if pred not in clusters:
        clusters[pred] = []
        clusters_count[pred] = 0
        
    clusters[pred].append(data_start[i][random.randint(0,len(data_start[i])-1)])
    clusters_count[pred]+=1

In [5]:
### clastering full text
from sentence_transformers import SentenceTransformer
import numpy as np
import random
from itertools import chain
from sklearn.cluster import AgglomerativeClustering
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [2]:
df = pd.read_csv("data/preproc_text700.csv")

In [3]:
ids = df.id_img.value_counts().to_dict()
indexs = [ids.get(i) for i in df.id_img.unique()]
data_proc = df.text_proc, indexs

In [9]:
# Corpus with example sentences
import time
start = time.time()
corpus_embeddings = embedder.encode(data_proc[0])

# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

#means tesor
corpus_embeddings = mean_tensor(torch.tensor(corpus_embeddings),data_proc[1])
print(time.time() - start)

334.0846381187439


In [19]:

from sklearn.cluster import MiniBatchKMeans
bs = 10000
knn = MiniBatchKMeans(n_clusters=1000,
                random_state=0,
                batch_size=bs)


In [ ]:
step = [0,0]
for _ in range(int(corpus_embeddings.shape[0]/bs)):
    step[1]+=bs
    knn = knn.partial_fit(corpus_embeddings[step[0]:step[1]])
    step[0]+=bs

In [24]:
step = [0,0]
pr = np.zeros(corpus_embeddings.shape[0])
for _ in range(int(corpus_embeddings.shape[0]/bs)):
    step[1]+=bs
    pr[step[0]:step[1]] = knn.predict(corpus_embeddings[step[0]:step[1]])
    step[0]+=bs

In [45]:
clustered_sentences = {}
for sentence_id, cluster_id in enumerate(pr):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(data_proc[0][sentence_id])